In [ ]:
! pip install kaggle

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json


In [ ]:
#https://www.kaggle.com/code/bettycxh06/rri-amplitudes/input
!kaggle datasets download -d bettycxh06/apnea-ecg-database

!kaggle kernels pull bettycxh06/rri-amplitudes


 97% 306M/315M [00:03<00:00, 109MB/s]
100% 315M/315M [00:04<00:00, 82.1MB/s]
Source code downloaded to /content/rri-amplitudes.ipynb


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
os.listdir(os.getcwd())

['.config',
 'apnea-ecg-database.zip',
 'kaggle.json',
 'rri-amplitudes.ipynb',
 'sample_data']

In [ ]:
!unzip apnea-ecg-database.zip

Archive:  apnea-ecg-database.zip
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/ANNOTATORS  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/RECORDS  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/SHA256SUMS.txt  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.apn  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.dat  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.hea  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.qrs  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01.xws  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.apn  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.hea  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.qrs  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01er.xws  
  inflating: apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0/a01r.apn  

In [ ]:
!pip install biosppy
!pip install wfdb

from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
import posixpath
import csv
import wfdb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.0/160.0 kB 3.7 MB/s eta 0:00:00


In [ ]:
import pickle
import sys
from concurrent.futures import ProcessPoolExecutor, as_completed

import biosppy.signals.tools as st
import numpy as np
import os
import wfdb
from biosppy.signals.ecg import correct_rpeaks, hamilton_segmenter
from scipy.signal import medfilt
from tqdm import tqdm

# PhysioNet Apnea-ECG dataset
# url: https://physionet.org/physiobank/database/apnea-ecg/
#base_dir = "../input/apnea-ecg-database/apnea-ecg-database-1.0.0"
base_dir = "apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0"
#base_dir = "physionet.org/files/apnea-ecg/1.0.0"



fs = 100
sample = fs * 60  # 1 min's sample points

before = 2  # forward interval (min)
after = 2  # backward interval (min)
hr_min = 20
hr_max = 300

num_worker = 35


def worker(name, labels):
    X = []
    y = []
    groups = []
    signals = wfdb.rdrecord(os.path.join(base_dir, name), channels=[0]).p_signal[:, 0]
    for j in tqdm(range(len(labels)), desc=name, file=sys.stdout):
        if j < before or \
                (j + 1 + after) > len(signals) / float(sample):
            continue
        signal = signals[int((j - before) * sample):int((j + 1 + after) * sample)]
        signal, _, _ = st.filter_signal(signal, ftype='FIR', band='bandpass', order=int(0.3 * fs),
                                        frequency=[3, 45], sampling_rate=fs)
        # Find R peaks
        rpeaks, = hamilton_segmenter(signal, sampling_rate=fs)
        rpeaks, = correct_rpeaks(signal, rpeaks=rpeaks, sampling_rate=fs, tol=0.1)
        if len(rpeaks) / (1 + after + before) < 40 or \
                len(rpeaks) / (1 + after + before) > 200:  # Remove abnormal R peaks signal
            continue
        # Extract RRI, Ampl signal
        rri_tm, rri_signal = rpeaks[1:] / float(fs), np.diff(rpeaks) / float(fs)
        rri_signal = medfilt(rri_signal, kernel_size=3)
        ampl_tm, ampl_siganl = rpeaks / float(fs), signal[rpeaks]
        hr = 60 / rri_signal
        # Remove physiologically impossible HR signal
        if np.all(np.logical_and(hr >= hr_min, hr <= hr_max)):
            # Save extracted signal
            X.append([(rri_tm, rri_signal), (ampl_tm, ampl_siganl)])
            y.append(0. if labels[j] == 'N' else 1.)
            groups.append(name)
    return X, y, groups


if __name__ == "__main__":
    apnea_ecg = {}

    names = [
        "a01", "a02", "a03", "a04", "a05", "a06", "a07", "a08", "a09", "a10",
        "a11", "a12", "a13", "a14", "a15", "a16", "a17", "a18", "a19", "a20",
        "b01", "b02", "b03", "b04", "b05",
        "c01", "c02", "c03", "c04", "c05", "c06", "c07", "c08", "c09", "c10"
    ]

    o_train = []
    y_train = []
    groups_train = []
    print('Training...')
    with ProcessPoolExecutor(max_workers=num_worker) as executor:
        task_list = []
        for i in range(len(names)):
            labels = wfdb.rdann(os.path.join(base_dir, names[i]), extension="apn").symbol
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_train.extend(X)
            y_train.extend(y)
            groups_train.extend(groups)

    print()



    answers = {}
    with open(os.path.join("event-2-answers"), "r") as f:
    #    with open(os.path.join(base_dir, "event-2-answers"), "r") as f:
        for answer in f.read().split("\n\n"):
            answers[answer[:3]] = list("".join(answer.split()[2::2]))

    names = [
        "x01", "x02", "x03", "x04", "x05", "x06", "x07", "x08", "x09", "x10",
        "x11", "x12", "x13", "x14", "x15", "x16", "x17", "x18", "x19", "x20",
        "x21", "x22", "x23", "x24", "x25", "x26", "x27", "x28", "x29", "x30",
        "x31", "x32", "x33", "x34", "x35"
    ]

    o_test = []
    y_test = []
    groups_test = []
    print("Testing...")
    with ProcessPoolExecutor(max_workers=num_worker) as executor:

        task_list = []
        for i in range(len(names)):
            labels = answers[names[i]]
            task_list.append(executor.submit(worker, names[i], labels))

        for task in as_completed(task_list):
            X, y, groups = task.result()
            o_test.extend(X)
            y_test.extend(y)
            groups_test.extend(groups)

    apnea_ecg = dict(o_train=o_train, y_train=y_train, groups_train=groups_train, o_test=o_test, y_test=y_test,
                     groups_test=groups_test)
    with open(os.path.join(base_dir, "apnea-ecg.pkl"), "wb") as f:
        pickle.dump(apnea_ecg, f, protocol=2)

    print("\nok!")

Training...
a17: 100%|██████████| 485/485 [44:39<00:00,  5.53s/it]

Testing...
x12: 100%|██████████| 527/527 [45:22<00:00,  5.17s/it]

ok!


In [ ]:
!pip install keras
!pip install tensorflow
#
#IMPORT LIBRARIES
import pickle
import numpy as np
import tensorflow
import os
from keras.callbacks import LearningRateScheduler,EarlyStopping
from keras.layers import Dense,Flatten,MaxPooling2D,Conv2D,BatchNormalization
from keras.regularizers import l2
from scipy.interpolate import splev, splrep
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
import keras
import matplotlib.pyplot as plt
from keras.layers import Conv1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.models import Model, load_model, save_model
from keras.layers import Input
import pandas as pd

#base_dir = "dataset"
base_dir = "apnea-ecg-database-1.0.0/apnea-ecg-database-1.0.0"
#base_dir = "physionet.org/files/apnea-ecg/1.0.0"

#------------------------------------------------------------------------------
# DEEP LEARNING MODELS NEED TO VECTORS OR MATRIX WITH SAME SIZE
# R-R INTERVALS DONT HAVE SAME SIZE, SO WE NEED TO INTERPOLATE VECTORS TO GET VECTORS WITH SAME SIZE.
# BASED ON OUR EXPERIENCE INTERPOLATION IN 3 HZ BETTER AND ACCURATE.
ir = 3 # INTERPOLATION RATE(3HZ)
time_range= 60 # 60-s INTERVALS OF ECG SIGNALS
weight=1e-3 #  WEIGHT L2 FOR REGULARIZATION(AVODING OVERFITTING PARAMETER)
#-----------------------------
# NORMALIZATION:
# DEEP LEARNING AND EVEN NEURAL NETWORKS INPUT SHOULD BE NORMALIZED:
# MIN-MAX METHOD APPLIED FOR SCALING:(Array-min(Array))/(max(Array)-min(Array))
scaler = lambda arr: (arr - np.min(arr)) / (np.max(arr) - np.min(arr))
#-----------------------------
# FIRSTLY WE PRE-PROCESSED OUR DATA IN "apnea-ecg.pkl" FILE
# IN PRE-PROCESSING SECTION WE EXTRACT R-R INTERVALS AND R-PEAK AMPLITUDES
# IN THIS PART WE LOAD THIS DATA AND INTERPOLATE AND CONCATE FOR FEEDING TO NETWORKS
def load_data():
    tm = np.arange(0, (before + 1 + after) * 60, step=1 / float(ir))

    with open(os.path.join(base_dir, "apnea-ecg.pkl"), 'rb') as f: # read preprocessing result
        apnea_ecg = pickle.load(f)
#-----------------
    x_train = []
    o_train, y_train = apnea_ecg["o_train"], apnea_ecg["y_train"]

    for i in range(len(o_train)):
        (rri_tm, rri_signal), (amp_tm, amp_signal) = o_train[i]
      # Curve interpolation
        rri_interp_signal = splev(tm, splrep(rri_tm, scaler(rri_signal), k=3), ext=1)
        amp_interp_signal = splev(tm, splrep(amp_tm, scaler(amp_signal), k=3), ext=1)
        x_train.append([rri_interp_signal, amp_interp_signal])
    x_train = np.array(x_train, dtype="float32")

    x_train = np.expand_dims(x_train,1)
    x_train=np.array(x_train, dtype="float32").transpose((0,3,1,2)) # convert to numpy format

    #return x_train_final, y_train
    return x_train, y_train

In [ ]:
!pip install Hyperactive
!pip install mealpy
!pip install tensorflow
!pip install optimizers
!pip install keras

#from keras.layers.rnn import GRU
from keras.layers import Reshape, Lambda
from keras.layers import GRU
from hyperactive import Hyperactive
from hyperactive.optimizers import HillClimbingOptimizer
from hyperactive.optimizers import RandomSearchOptimizer
from mealpy.swarm_based.GWO import OriginalGWO

from tensorflow.keras import optimizers
from keras.optimizers import Adam
import threading
import concurrent.futures
from threading import Thread
import time
from tabnanny import verbose
from mealpy.swarm_based.HBA import OriginalHBA

from tensorflow.python.keras import optimizers
#from keras.optimizers import RandomSearchOptimizer

import warnings
warnings.filterwarnings('ignore')


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.1/64.1 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 378.0/378.0 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 28.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for optimizers: filename=Optimizers-0.1-py3-none-any.whl size=17005 sha256=bca83c64f2b2ff9ea2fe86f460f4e8060df24b5409350933362a3fcdbe1a4309
  Stored in directory: /root/.cache/pip/wheels/21/85/73/6114e547e45ab65253df3cba8144782286634fa9773a5a7828
Successfully built optimizers


In [ ]:
#print(x_train.shape, y_train.shape)
#x_train= np.reshape(x_train,(x_train.shape[0],450,2,2))    #90---450?
#x_train= np.concatenate((x_train[:,:450,:,0],x_train[:,:450,:,1]),axis=1)
#x_train= x_train.transpose((0,2,1))
#print(x_train.shape, y_train.shape)
##(16709, 900, 1, 2) (16709, 2)
##(16709, 2, 900) (16709, 2)


In [ ]:
# BHA
###########################
def fitness_function(position):
    learning_rate,n1,n2,n3,n4,epochs = position[0],position[1],position[2],position[3],position[4],position[5]
    n1,n2,n3,n4,epochs=int(n1),int(n2),int(n3),int(n4),int(epochs)

    print(learning_rate,n1,n2,n3,n4,epochs)

#def create_model(weight=1e-3):
    model= Sequential()
    model.add(GRU(n1, return_sequences=True, input_shape=(2,900)))    #(2,180)))
    model.add(GRU(n2, return_sequences=True))
    model.add(GRU(n3, return_sequences=True))

    model.add(Flatten())
    model.add(Dense(n4, activation="relu"))
    model.add(Dense(2, activation="softmax"))

    return learning_rate,n1,n2,n3,n4,epochs


    optimizer =keras.optimizers.Adam(lr=learning_rate)
    print("Learning rate: ", learning_rate)
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=['accuracy'])
    #model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=['accuracy'])

    history=model.fit(X1, Y1, batch_size=128, epochs=int(epochs), validation_data=(x_val, y_val),
                       callbacks=[callback1,lr_scheduler])


    #history = model.fit(x_train,y_train, epochs=int(epochs),batch_size=32, validation_data=(x_test, y_test))
    return history.history['val_accuracy'][-1]
    #loss, accuracy = model.evaluate(x_test, y_test)

    #return model
#------------------------------------------------------------------------------
# Define learning rate schedule for preventing overfitting in deep learning methods:
def lr_schedule(epochs, learning_rate):
   if epochs > 70 and \
           (epochs - 1) % 10 == 0:
        learning_rate *= 0.1
   print("Learning rate: ", learning_rate)
   return learning_rate

#------------------------------------------------------------------------------

        # we used k-fold cross-validation for more reliable experiments:
   kfold = StratifiedKFold(n_splits=5, shuffle=True,random_state=7)
   cvscores = []
   ACC=[]
   SN=[]
   SP=[]
   F2=[]
   print("train num:", len(y_train))
   print("test num:", len(y_test))


    # separate train& test and then compile model
  # for train, test in kfold.split(x_train, y_train.argmax(1)):
     #model = create_model()
     #model.summary()

     # Compile and evaluate model:
if __name__ == "__main__":
    # loading Data:
    x_train, y_train = load_data()
    # we have labels(Y) in a binary way 0 for normal and 1 for apnea patients
    # we want to classify data into 2-class so we changed y in a categorical way:
    y_train = tf.keras.utils.to_categorical(y_train, num_classes=2)
    # we used k-fold cross-validation for more reliable experiments:
    kfold = StratifiedKFold(n_splits=5, shuffle=True,random_state=7)


    # separate train& test and then compile model
    ##for train, test in kfold.split(x_train, y_train.argmax(1)):
     #model = create_model()
     #model.summary()

    # define callback for early stopping:
    lr_scheduler = LearningRateScheduler(lr_schedule)
    callback1 = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    x_train= np.reshape(x_train,(x_train.shape[0],450,2,2))    #90---450?
    x_train= np.concatenate((x_train[:,:450,:,0],x_train[:,:450,:,1]),axis=1)
    x_train= x_train.transpose((0,2,1))
    #10% of Data used for validation:
    X1,x_val,Y1,y_val=train_test_split(x_train,y_train,test_size=0.10)

##################################################################
#pso = Pso(swarmsize=4,maxiter=14)
# n,sf,sp,l
##bp,value = pso.run(func,[1,2,2,2],[16,8,4,4])
#v = func(bp);
##################################################################
problem_dict1 = {
    "fit_func": fitness_function,
    'lb':[0.001,50,10,9,4,30], # Lower bound of our parameters
    'ub':[0.1,80,40,19,8,200], # upper bound of our parameters
    "minmax": "min",
    "obj_weights": [0.4,1,1,1,1,1]               # Define it or default value will be [1, 1, 1]   [0.4, 0.1, 0.5,1,1,1,1,1]
    #"mode":"thread"
    #"amend_position": amend_position

                }

rng = np.random.default_rng()
epoch = 100
pop_size =50

#def thread_function(k):

  #run=k+1
  #print("Run Number",run)
  #Optimizer = MHoneyBadgerAlgorithm(search_config, n_iter=100, n_part=10, metric='accuracy', cv=10, h_beta=6.0, h_c=2.0,run=run)
model = OriginalGWO(epoch, pop_size)
  #t1 = time.time()

#best_position, best_fitness = model.solve(problem_dict1)
best_position, best_fitness= model.solve(problem_dict1)
#best_position, best_fitness= model.solve(mode="thread")
v = fitness_function(best_position);
#print(f"Solution: {best_position}, Fitness: {best_fitness}")
#return v
print(f"Solution: {best_position}, Fitness: {v}")

0.018681680823935325 63 38 11 4 70


INFO:mealpy.swarm_based.GWO.OriginalGWO:Solving 6-objective optimization problem with weights: [0.4 1.  1.  1.  1.  1. ].


0.05788326699567425 61 28 12 5 54
0.07406470416715262 54 30 15 6 124
0.00427092430386331 74 29 9 5 64
0.0463369418237985 61 24 14 5 139
0.05369020860577551 61 32 14 6 142
0.04848018754043077 58 16 11 5 175
0.07012235840062798 53 38 16 4 177
0.08418524174328079 66 17 14 4 160
0.04183991926235848 74 11 10 6 136
0.08113654142170279 65 18 11 4 159
0.004325479171033428 53 24 14 6 78
0.011802133247285126 74 14 13 4 30
0.09814283206900479 76 39 18 7 170
0.06341515270741001 79 11 14 7 53
0.008516957418228163 57 34 17 6 194
0.013005065831490067 72 13 9 5 174
0.057093563707384255 59 36 12 5 57
0.0617836132034146 59 20 10 4 146
0.022359970189161617 59 34 11 4 33
0.058185179103955056 73 18 16 5 51
0.07609643991142091 63 16 14 6 121
0.03788683579730572 67 22 15 5 90
0.08991899149766196 63 38 16 5 66
0.08313540465683045 73 37 12 4 123
0.05231246974174878 54 34 14 4 41
0.03482653985871339 68 24 15 7 92
0.09746802150036732 58 17 18 5 67
0.023932489981237752 68 28 9 6 162
0.09300659457195688 55 21 12 7

INFO:mealpy.swarm_based.GWO.OriginalGWO:>Problem: P, Epoch: 1, Current best: 104.00368830999042, Global best: 104.00368830999042, Runtime: 34.62187 seconds


0.001 80 18 9 6 30
0.03784291351096055 80 10 9 5 30
0.04744031598490924 50 14 14 4 39
0.027315928118484083 50 10 12 5 34
0.016647033709392165 63 10 9 4 30
0.012300820944796502 50 22 10 4 30
0.017901096034264943 66 11 9 4 30
0.001 58 10 11 4 43
0.001 56 11 12 4 30
0.026498886918457015 50 15 13 4 43
0.05409817623527743 80 10 9 4 34
0.011858809733990672 63 10 11 4 39
0.03679189966507974 68 17 9 4 48
0.005702701672704549 75 16 9 6 56
0.01223076930605241 53 10 13 4 30
0.004489562302354766 50 10 12 4 30
0.01943202733978443 50 12 13 4 43
0.0310709818277673 54 10 9 4 30
0.009772644327529471 50 14 9 4 30
0.029537082987727673 50 10 11 6 33
0.0115926807091465 50 14 9 4 45
0.013678511195381183 69 15 9 4 55
0.010964612032235409 50 10 9 4 47
0.001 50 17 9 4 33
0.009737879992381848 80 16 9 4 30
0.001 58 10 9 6 30
0.02419433244988904 50 10 9 4 30
0.02496318948968183 76 10 9 4 30
0.08989621222191517 50 10 17 5 100
0.001 50 21 13 4 42
0.001 69 14 19 4 80
0.020842730965660725 50 10 9 4 30
0.0198270237665

...

0.001001428285120857 50 10 9 4 30
0.0010013494674600252 50 10 9 4 30
0.0010019924310416452 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.0010032292600730683 50 10 9 4 30
0.001001677648701377 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.0010009576991848916 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.0010048757750919827 50 10 9 4 30
0.0010058856856789194 50 10 9 4 30
0.001 50 10 9 4 30
0.0010032175683477743 50 10 9 4 30
0.0010036984669847702 50 10 9 4 30
0.0010030741114443688 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.0010018641798923873 50 10 9 4 30
0.001 50 10 9 4 30
0.0010034663931252463 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.0010067902088862115 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001005031830202978 50 10 9 4 30
0.0010043639512820706 50 10 9 4 30
0.001 

INFO:mealpy.swarm_based.GWO.OriginalGWO:>Problem: P, Epoch: 99, Current best: 103.0004, Global best: 103.0004, Runtime: 34.23562 seconds


0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30
0.001 50 10 9 4 30


INFO:mealpy.swarm_based.GWO.OriginalGWO:>Problem: P, Epoch: 100, Current best: 103.0004, Global best: 103.0004, Runtime: 34.18833 seconds


0.001 50 10 9 4 30
Solution: [1.00000000e-03 5.08460949e+01 1.00000000e+01 9.03645867e+00
 4.55018114e+00 3.00000000e+01], Fitness: (0.001, 50, 10, 9, 4, 30)
